In [1]:
# Setup for Advanced Modeling
"""
Advanced Modeling with SMOTE and XGBoost
========================================
Imporving fraud detection with advanced techniques
"""

import sys
sys.path.append('..')
import os
os.chdir('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Import our modules
from src.data.loader import DataLoader
from src.features.engineer import FeatureEngineer

print("Imports complete")

Imports complete
